# Data Processing Steps
## MQTT
Using the `capture.py` scripts located in the `mqtt` subdirectory, we subscribed to every topic available on the given server and we listened for messages for a 30 minutes timeframe. The script then saved the output of the capture to a `.json` file ready to be processed.

### Step 1: Removing Duplicates
First we want to remove duplicates from both MQTT and COAP captures, and then we want to merge the results.
To process the MQTT capture, which is located at `mqtt/data.json`, we'll use the following code block:

In [2]:
from json import load


def remove_duplicates(data):
    res = []
    for msg in data:
        if msg not in res:
            res.append(msg)
    return res

# opening file located at mqtt/data.json
messages = dict()
filepath = "mqtt/data.json"
with open(filepath, "r") as f:
    messages = load(f)["messages"]

print("Number of messages captured: %d" % len(messages))
messages = remove_duplicates(messages)
print("Number of messages captured after removing duplicates: %d" % len(messages))

Number of messages captured: 3202
Number of messages captured after removing duplicates: 289


### Step 2: Filtering out bad coordinates
Now we can select coordinates which are actually useful. For this task we choose to remove all messages with a payload which either is not a coordinate or has out of bounds coordinates.

In [3]:
from json import load
from re import findall, search

print(len(messages))
def filter_data(data):
    res = []
    bounds = [0, 10]
    for msg in data:
        if msg["payload"].count("|") == 6:

            # the first 7 characters are the two decimal values with
            # one decimal digit, separated by a comma that we want to
            # parse as coordinates
            coords_regex = r"(\d+\.\d+),(\d+\.\d+)"
            if search(coords_regex, msg["payload"]) is None:
                continue
            x, y = findall(coords_regex, msg["payload"])[0]
            # now we check if either coordinate is out of bound
            if (
                (float(x) >= bounds[0] and float(x) <= bounds[1])
                and (float(y) >= bounds[0] or float(y) <= bounds[1])
            ):
                res.append(msg)
    return res

messages = filter_data(messages)
messages = remove_duplicates(messages)
print("Number of messages captured after removing junk values: %d" % len(messages))

289
Number of messages captured after removing junk values: 15


### Step 3: Manual Cleanup
Now that we have only valid coordinates, we can manually select just the ones that actually belong to Dory.

In [5]:
from rich import print_json
for item in messages:
    print_json(data=item)

{
  "topic": "/wierd/topic/isnt/it",
  "payload": "4.0,6.0|-63,-62,-63,-62,-63|-61,-60,-61,-65,-61|-62,-60,-62,-61,-61|-65,-63,-65,-63,-64|-59,-57,-59,-57,-58|-63,-62,-63,-62,-63",
  "qos": 0,
  "retain": 0
}

{
  "topic": "find/this/hidden/dory/here",
  "payload": "0.0,8.0|-63,-60,-60,-59,-64|-51,-49,-47,-48,-53|-64,-62,-67,-61,-66|-67,-66,-64,-63,-68|-59,-57,-58,-56,-61|-65,-63,-64,-62,-66",
  "qos": 0,
  "retain": 0
}

{
  "topic": "/dory/has/been/here",
  "payload": "6.0,4.0|-63,-61,-65,-63,-64|-64,-63,-64,-65,-65|-58,-56,-58,-59,-59|-62,-59,-61,-61,-62|-62,-60,-61,-62,-62|-65,-61,-63,-63,-64",
  "qos": 0,
  "retain": 0
}

{
  "topic": "find/this/hidden/dory/there",
  "payload": "0.0,10.0|-69,-64,-67,-65,-65|-19,-24,-25,-25,-25|-60,-65,-69,-66,-66|-62,-67,-70,-68,-68|-53,-58,-61,-59,-59|-59,-64,-67,-65,-65",
  "qos": 0,
  "retain": 0
}

{
  "topic": "hey/dory/where/are/u",
  "payload": "2.0,8.0|-61,-59,-63,-67,-65|-52,-53,-54,-58,-56|-61,-59,-64,-67,-65|-63,-62,-66,-69,-68|-54,-55,-56,-60,-58|-61,-59,-63,-67,-65",
  "qos": 0,
  "retain": 0
}

...

{
  "topic": "nemo/tracks/dory",
  "payload": "10.0,4.0|-66,-67,-65,-65,-71|-67,-69,-65,-66,-71|-62,-63,-60,-61,-66|-58,-59,-56,-57,-62|-64,-64,-64,-62,-68|-61,-62,-60,-60,-66",
  "qos": 0,
  "retain": 0
}

{
  "topic": "marlin/and/nemo/are/stalkers",
  "payload": "10.0,8.0|-66,-64,-60,-66,-69|-64,-62,-58,-64,-67|-63,-62,-57,-63,-66|-62,-60,-56,-62,-65|-58,-57,-53,-58,-61|-50,-48,-44,-50,-53",
  "qos": 0,
  "retain": 0
}

{
  "topic": "no/nemo/here/discard/this",
  "payload": "6.6,6.6|-23,-45,-63,-32,-65|-13,-12,-94,-58,-106|-66,-66,-666,-66,-66|-55,-55,-55,-55,-55|-77,-77,-777,-77,-77|-21,-59,-33,-67,-121",
  "qos": 0,
  "retain": 0
}

{
  "topic": "dory/has/been/here/too",
  "payload": "6.0,6.0|-64,-65,-63,-64,-63|-62,-64,-61,-62,-61|-61,-62,-60,-61,-60|-62,-64,-61,-62,-61|-57,-59,-57,-57,-57|-60,-62,-59,-60,-59",
  "qos": 0,
  "retain": 0
}

{
  "topic": "find/this/hidden/dory/",
  "payload": "0.0,0.0|-27,-30,-29,-25,-23|-67,-70,-68,-65,-63|-61,-65,-63,-59,-57|-67,-71,-68,-65,-63|-68,-71,-69,-66,-64|-70,-73,-72,-68,-66",
  "qos": 0,
  "retain": 0
}

By printing them we can see that some are clearly misleading. We can proceed to filter out those entries.

In [7]:
from rich import print_json
from json import dumps, dump, load

blacklist = [
    "/wierd/topic/isnt/it",
    "cant/be/a/valid/entry",
    "hi/nemo/wassup/",
    "marlin/checked/this/entry",
    "nemo/tracks/dory",
    "marlin/and/nemo/are/stalkers",
    "no/nemo/here/discard/this"
]
for item in messages:
    if item["topic"] in blacklist:
        messages.remove(item)

with open("mqtt_data.json", "w") as f:
    for item in messages:
        print_json(data=item)
    dump(messages, f, indent=4)

{
  "topic": "find/this/hidden/dory/here",
  "payload": "0.0,8.0|-63,-60,-60,-59,-64|-51,-49,-47,-48,-53|-64,-62,-67,-61,-66|-67,-66,-64,-63,-68|-59,-57,-58,-56,-61|-65,-63,-64,-62,-66",
  "qos": 0,
  "retain": 0
}

{
  "topic": "/dory/has/been/here",
  "payload": "6.0,4.0|-63,-61,-65,-63,-64|-64,-63,-64,-65,-65|-58,-56,-58,-59,-59|-62,-59,-61,-61,-62|-62,-60,-61,-62,-62|-65,-61,-63,-63,-64",
  "qos": 0,
  "retain": 0
}

{
  "topic": "find/this/hidden/dory/there",
  "payload": "0.0,10.0|-69,-64,-67,-65,-65|-19,-24,-25,-25,-25|-60,-65,-69,-66,-66|-62,-67,-70,-68,-68|-53,-58,-61,-59,-59|-59,-64,-67,-65,-65",
  "qos": 0,
  "retain": 0
}

{
  "topic": "hey/dory/where/are/u",
  "payload": "2.0,8.0|-61,-59,-63,-67,-65|-52,-53,-54,-58,-56|-61,-59,-64,-67,-65|-63,-62,-66,-69,-68|-54,-55,-56,-60,-58|-61,-59,-63,-67,-65",
  "qos": 0,
  "retain": 0
}

{
  "topic": "find/this/hidden/dory/over/there",
  "payload": "2.0,0.0|-55,-51,-50,-50,-57|-69,-65,-64,-64,-71|-58,-55,-53,-54,-60|-67,-63,-62,-64,-69|-69,-66,-64,-64,-71|-71,-67,-66,-66,-73",
  "qos": 0,
  "retain": 0
}

{
  "topic": "please/dory/come/back/home",
  "payload": "4.0,2.0|-60,-60,-63,-58,-59|-66,-66,-69,-64,-65|-54,-54,-58,-53,-53|-63,-63,-66,-61,-62|-65,-65,-68,-63,-64|-67,-67,-70,-65,-66",
  "qos": 0,
  "retain": 0
}

{
  "topic": "nemo/tracks/dory",
  "payload": "10.0,4.0|-66,-67,-65,-65,-71|-67,-69,-65,-66,-71|-62,-63,-60,-61,-66|-58,-59,-56,-57,-62|-64,-64,-64,-62,-68|-61,-62,-60,-60,-66",
  "qos": 0,
  "retain": 0
}

{
  "topic": "dory/has/been/here/too",
  "payload": "6.0,6.0|-64,-65,-63,-64,-63|-62,-64,-61,-62,-61|-61,-62,-60,-61,-60|-62,-64,-61,-62,-61|-57,-59,-57,-57,-57|-60,-62,-59,-60,-59",
  "qos": 0,
  "retain": 0
}

{
  "topic": "find/this/hidden/dory/",
  "payload": "0.0,0.0|-27,-30,-29,-25,-23|-67,-70,-68,-65,-63|-61,-65,-63,-59,-57|-67,-71,-68,-65,-63|-68,-71,-69,-66,-64|-70,-73,-72,-68,-66",
  "qos": 0,
  "retain": 0
}